# Shallow fault dip Vs rake

In [ ]:
%matplotlib inline
import re
import os
import sys
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

sys.path.append('/Users/mpagani/Repos/original/oq-engine/')
sys.path.append('/Users/mpagani/Repos/oq-man/')

import oqman.man.model as model

from oqman.man.single.faults import get_lengths
from oqman.hazardlib.const import TRT

from openquake.hazardlib.geo.surface import SimpleFaultSurface
from openquake.hazardlib.source import SimpleFaultSource, CharacteristicFaultSource

## List of models to be analysed

In [ ]:
path = '/Users/mpagani/NC/Hazard_Charles/Documents/Presentations/2017/0417_SSA/notebooks/pkl/*.pkl'
modell = set(['als07', 'aus12', 'ca10h', 'ca10r', 'cub03', 'ear16', 'em16a',
              'em16f', 'emc15', 'lea02', 'nzl10', 'res12', 'sar16', 'sea07',
              'soa10', 'twn15', 'usa08'])
modell = set(['aus12', 'ca10h', 'ca10r', 'cub03', 'ear16', 'em16a',
              'em16f', 'emc15', 'eu13f', 'lea02', 'nzl10', 'res12', 'sar16', 'sea07',
              'soa10', 'twn15', 'usa08'])
#modell = set(['twn15'])
#modell = set(['ca10r'])

In [ ]:
#path = '/Users/mpagani/Prj/2017/global_modelling/results/pkl/*.pkl'
#modell = set(['arm17', 'em16f'])

## Read data for the different models

In [ ]:
modd = {}
for fname in glob.glob(path):
    slist = re.split('\.',  os.path.basename(fname)) 
    if re.search('[a-z]*[0-9]*\.pkl', fname) and slist[0] in modell:
        print (fname, end=' ')
        mod, points, info, sidx = model.load(fname) 
        print ('done')
        modd[slist[0]] = {'model': mod, 'info': info}

In [ ]:
def get_surface_from_sfs(src, mesh_spacing):
    return SimpleFaultSurface.from_fault_data(src.fault_trace, 
                                              src.upper_seismogenic_depth,
                                              src.lower_seismogenic_depth, 
                                              src.dip, mesh_spacing)

## Plotting

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1, 1, 1)

mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14
        
#fltr = 'Stable Continental Crust|Stable Continental Region|Stable Shallow Crust'
fltr = 'Active Shallow Crust'

mesh_spacing = 2.
src_types = (SimpleFaultSource, CharacteristicFaultSource)

for key in sorted(modd.keys()):
    print (key)

    rake = []
    dip = []

    # Check if the model contains faults sources
    if ('SimpleFaultSource' in modd[key]['info']['srcs_mmax'] or 
        'CharacteristicFaultSource' in modd[key]['info']['srcs_mmax']): 

        for src in modd[key]['model']:
            srfc = None
            if isinstance(src, SimpleFaultSource):
                srfc = get_surface_from_sfs(src, mesh_spacing)
            elif isinstance(src, CharacteristicFaultSource):
                srfc = src.surface
            if isinstance(src, src_types) and srfc is not None:
                rake.append(src.rake)
                dip.append(srfc.get_dip())
        plt.plot(rake, dip, 'o')
        
plt.xlabel('Rake', fontsize=14)
plt.ylabel('Dip', fontsize=14)
plt.grid(linestyle='--')
plt.xlim([-92, 92])